In [1]:
import numpy as np
import scipy as sp
from pandas import Series, DataFrame
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

pd.set_option('display.max_columns', 1000000)
pd.set_option('display.max_rows', 1000000)



In [2]:
ls

all.js           LEO_sun.csv            satellites.css
core             MEO_Elliptical.csv     satellites.ipynb
CSatellites.csv  MEO.html               sLEO.html
d3.v3.js         MEO.js                 sLEO.js
GEO.csv          oLEO.html              table.html
GEO.html         oLEO.js                UCS_Satellite_Database_9-1-2017.tsv
GEO.js           README.md              world-50m.json
index.html       satellite_map.png
LEO_other.csv    satellite_plots.ipynb


In [3]:
#Importing data and removing empty columns
#data = pd.read_table('Fsatellites.tsv', encoding = "ISO-8859-1") old data
data = pd.read_table('UCS_Satellite_Database_9-1-2017.tsv', encoding = "ISO-8859-1") # newdata
print(data.shape)

for col in data.columns:
    if 'Unnamed' in col:
        del data[col]
        
for col in data.columns:
    if 'Source' in col:
        del data[col]
data.head()

(1738, 254)


,"Name of Satellite, Alternate Names",Country/Org of UN Registry,Country of Operator/Owner,Operator/Owner,Users,Purpose,Detailed Purpose,Class of Orbit,Type of Orbit,Longitude of GEO (degrees),Perigee (km),Apogee (km),Eccentricity,Inclination (degrees),Period (minutes),Launch Mass (kg.),Dry Mass (kg.),Power (watts),Date of Launch,Expected Lifetime,Contractor,Country of Contractor,Launch Site,Launch Vehicle,COSPAR Number,NORAD Number,Comments
0,Aalto-1,NR (10/17),Finland,University of Aalto,Civil,Technology Development,NaN,LEO,NaN,0.00,497,517,0.001450,97.45,94.7,5,NaN,4.5,6/23/2017,2 yrs.,University of Aalto,Finland,Satish Dhawan Space Centre,PSLV,2017-036L,42775,Technology development and education.
1,AAUSat-4,NR (12/16),Denmark,University of Aalborg,Civil,Earth Observation,Automatic Identification System (AIS),LEO,Sun-Synchronous,0.00,442,687,0.017700,98.20,95.9,1,NaN,NaN,4/25/2016,NaN,University of Aalborg,Denmark,Guiana Space Center,Soyuz 2.1a,2016-025E,41460,Carries AIS system.
2,"ABS-2 (Koreasat-8, ST-3)",NR,Multinational,Asia Broadcast Satellite Ltd.,Commercial,Communications,NaN,GEO,NaN,75.00,"35,778","35,793",0.000178,0.08,1436.03,"6,330",NaN,"16,000",2/6/2014,15 yrs.,Space Systems/Loral,USA,Guiana Space Center,Ariane 5 ECA,2014-006A,39508,"32 C-band, 51 Ku-band, and 6 Ka-band transpond..."
3,ABS-2A,NR (12/16),Multinational,Asia Broadcast Satellite Ltd.,Commercial,Communications,NaN,GEO,NaN,-75.00,"35,700","35,700",0.000000,0.00,1436.1,"1,800",NaN,NaN,6/15/2016,15 yrs.,Boeing Satellite Systems,USA,Cape Canaveral,Falcon 9,2016-038A,41588,NaN
4,"ABS-3 (Agila 2, Mabuhay 1)",Philippines,Multinational,Asia Broadcast Satellite Ltd.,Commercial,Communications,NaN,GEO,NaN,146.06,"35,769","35,802",0.000391,0.05,1436.07,"3,775","1,800","9,000",8/19/1997,15 yrs.,Space Systems/Loral,USA,Xichang Satellite Launch Center,Long March CZ3B,1997-042A,24901,Most powerful telecommunications satellite in ...


In [4]:
#Renaming columns
data.columns.values

array(['Name of Satellite, Alternate Names', 'Country/Org of UN Registry',
       'Country of Operator/Owner', 'Operator/Owner', 'Users', 'Purpose',
       'Detailed Purpose', 'Class of Orbit', 'Type of Orbit',
       'Longitude of GEO (degrees)', 'Perigee (km)', 'Apogee (km)',
       'Eccentricity', 'Inclination (degrees)', 'Period (minutes)',
       'Launch Mass (kg.)', 'Dry Mass (kg.)', 'Power (watts)',
       'Date of Launch', 'Expected Lifetime', 'Contractor',
       'Country of Contractor', 'Launch Site', 'Launch Vehicle',
       'COSPAR Number', 'NORAD Number', 'Comments'], dtype=object)

In [5]:
data.columns=['name', 'country_reg', 'country', 'owner', 'users',
              'purpose', 'description', 'CLO', 'TOO', 'longitude',
              'perigee_km', 'apogee_km', 'eccentricity', 'inclination', 
              'period_minutes', 'launch_mass', 'dry_mass', 'power_watts',
              'launch_date', 'expected_lifetime_years', 'contractor', 
              'contractor_country', 'launch_site', 'launch_vehicle',
              'COSPAR', 'NORAD', 'comments']


In [6]:
data['CLO'] = data['CLO'].replace('LEO ', 'LEO')

In [7]:
#Standardizing expected_lifetime values
data_NoC = data.replace(to_replace=',', value='', regex=True)
data_NoC['expected_lifetime_years'] = data_NoC['expected_lifetime_years'].replace(to_replace="(yrs\.|yr\.|hrs\.|trs)", value='', regex=True)
data_NoC['expected_lifetime_years'] = data_NoC['expected_lifetime_years'].replace(to_replace="\.?[0-9]*-", value='', regex=True)
data_NoC['expected_lifetime_years'] = data_NoC['expected_lifetime_years'].replace(to_replace="\+", value='', regex=True)
pd.unique(data_NoC['expected_lifetime_years'])

array(['2 ', nan, '15 ', '12 ', '14 ', '3 ', '5 ', '.25 ', '.5 ', '10 ',
       '14', '1 ', '8 ', '7 ', '1.5 ', '13 ', '6 ', '9 ', '18 ', '16 ',
       '11 ', '15  ', '20 ', '4 ', '10 ys.', '17 ', '30 ', '14.5 ',
       '7.25 ', ' 3 ', '11.5 ', '2.5 ', '12.6 ', '.5  2 ', '4.5 '], dtype=object)

In [8]:
data['expected_lifetime_years'].isnull().sum()

709

In [9]:
data_NoC['expected_lifetime_years'].isnull().sum()

709

In [10]:
#Convert data
data_converted = data_NoC.convert_objects(convert_dates=True, convert_numeric=True,)
data = data_converted


/usr/lib/python3.6/site-packages/ipykernel/__main__.py:2: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  from ipykernel import kernelapp as app


In [11]:
data['launch_date'] = pd.to_datetime(data['launch_date'])
data['expected_lifetime_years'].isnull().sum()

711

In [12]:
pd.unique(data.expected_lifetime_years)

array([  2.  ,    nan,  15.  ,  12.  ,  14.  ,   3.  ,   5.  ,   0.25,
         0.5 ,  10.  ,   1.  ,   8.  ,   7.  ,   1.5 ,  13.  ,   6.  ,
         9.  ,  18.  ,  16.  ,  11.  ,  20.  ,   4.  ,  17.  ,  30.  ,
        14.5 ,   7.25,  11.5 ,   2.5 ,  12.6 ,   4.5 ])

In [13]:
#Check for correct dtype
datatypes = data.columns.to_series().groupby(data.dtypes).groups
datatypes

{dtype('int64'): Index(['perigee_km', 'apogee_km', 'NORAD'], dtype='object'),
 dtype('float64'): Index(['longitude', 'eccentricity', 'inclination', 'period_minutes',
        'launch_mass', 'dry_mass', 'power_watts', 'expected_lifetime_years'],
       dtype='object'),
 dtype('<M8[ns]'): Index(['launch_date'], dtype='object'),
 dtype('O'): Index(['name', 'country_reg', 'country', 'owner', 'users', 'purpose',
        'description', 'CLO', 'TOO', 'contractor', 'contractor_country',
        'launch_site', 'launch_vehicle', 'COSPAR', 'comments'],
       dtype='object')}

In [14]:
##Fix missing launch dates
data.set_value(1144, 'launch_date', "2015-01-01");
data.set_value(1145, 'launch_date', "2015-01-01");
data.set_value(1198, 'launch_date', "2013-01-01");


In [15]:
#Create columns to isolate launch year/month/day
data['launch_year'], data['launch_month'], data['launch_day'] = data['launch_date'].dt.year, data['launch_date'].dt.month, data['launch_date'].dt.day
data.columns



Index(['name', 'country_reg', 'country', 'owner', 'users', 'purpose',
       'description', 'CLO', 'TOO', 'longitude', 'perigee_km', 'apogee_km',
       'eccentricity', 'inclination', 'period_minutes', 'launch_mass',
       'dry_mass', 'power_watts', 'launch_date', 'expected_lifetime_years',
       'contractor', 'contractor_country', 'launch_site', 'launch_vehicle',
       'COSPAR', 'NORAD', 'comments', 'launch_year', 'launch_month',
       'launch_day'],
      dtype='object')

In [16]:
data.head()

,name,country_reg,country,owner,users,purpose,description,CLO,TOO,longitude,perigee_km,apogee_km,eccentricity,inclination,period_minutes,launch_mass,dry_mass,power_watts,launch_date,expected_lifetime_years,contractor,contractor_country,launch_site,launch_vehicle,COSPAR,NORAD,comments,launch_year,launch_month,launch_day
0,Aalto-1,NR (10/17),Finland,University of Aalto,Civil,Technology Development,NaN,LEO,NaN,0.00,497,517,0.001450,97.45,94.70,5.0,NaN,4.5,2017-06-23,2.0,University of Aalto,Finland,Satish Dhawan Space Centre,PSLV,2017-036L,42775,Technology development and education.,2017,6,23
1,AAUSat-4,NR (12/16),Denmark,University of Aalborg,Civil,Earth Observation,Automatic Identification System (AIS),LEO,Sun-Synchronous,0.00,442,687,0.017700,98.20,95.90,1.0,NaN,NaN,2016-04-25,NaN,University of Aalborg,Denmark,Guiana Space Center,Soyuz 2.1a,2016-025E,41460,Carries AIS system.,2016,4,25
2,ABS-2 (Koreasat-8 ST-3),NR,Multinational,Asia Broadcast Satellite Ltd.,Commercial,Communications,NaN,GEO,NaN,75.00,35778,35793,0.000178,0.08,1436.03,6330.0,NaN,16000.0,2014-02-06,15.0,Space Systems/Loral,USA,Guiana Space Center,Ariane 5 ECA,2014-006A,39508,32 C-band 51 Ku-band and 6 Ka-band transponder...,2014,2,6
3,ABS-2A,NR (12/16),Multinational,Asia Broadcast Satellite Ltd.,Commercial,Communications,NaN,GEO,NaN,-75.00,35700,35700,0.000000,0.00,1436.10,1800.0,NaN,NaN,2016-06-15,15.0,Boeing Satellite Systems,USA,Cape Canaveral,Falcon 9,2016-038A,41588,NaN,2016,6,15
4,ABS-3 (Agila 2 Mabuhay 1),Philippines,Multinational,Asia Broadcast Satellite Ltd.,Commercial,Communications,NaN,GEO,NaN,146.06,35769,35802,0.000391,0.05,1436.07,3775.0,1800.0,9000.0,1997-08-19,15.0,Space Systems/Loral,USA,Xichang Satellite Launch Center,Long March CZ3B,1997-042A,24901,Most powerful telecommunications satellite in ...,1997,8,19


In [17]:
#Remove nickaname from name column to its own column
data['nickname'] = data['name'].str.extract('(\(.*\))')
data['name'] = data['name'].replace(to_replace="\(.*\)", value='', regex=True)


/usr/lib/python3.6/site-packages/ipykernel/__main__.py:2: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  from ipykernel import kernelapp as app


In [18]:
#Fix and standardize longitudinal degrees to 360 for GEO orbits
data['longitude'][data['longitude'] > 180] = data['longitude'] - 360
data['longitude'][data['longitude'] < 0 ] = data['longitude'] + 360

/usr/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/usr/lib/python3.6/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [19]:
#Adding arbitrary longitudes to non-GEO orbits for successional calculations
##GEOsynchronous orbits appear stationary at certian longitudes from earths surface
data['longitude'][data['longitude'] == 0 ] = data['longitude'].apply(lambda v: np.random.randint(360))

/usr/lib/python3.6/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [20]:
#Calculate radians for GEO orbit longitudes
data['radians'] = np.deg2rad(data['longitude'])

In [21]:
##Create constant variables
#(mean)radius of earth km
earthRad = 6371
#Mass of earth times gravitational constant
GM = 398600.4


In [22]:
#Include earth's radius in apogee and perigee values for elliptical calculations
data['apogeeR'] = data['apogee_km'] + 6371
data['perigeeR'] = data['perigee_km'] + 6371

In [23]:
#Calculate the major/semimajor axis of orbits
data['majorA'] = data.apogeeR + data.perigeeR
data['semimajorA'] = data['majorA']/2


In [24]:
#The distance between earth and the empty focus of the ellipse
data['fDistance'] = data['majorA'] - data['perigeeR']*2

In [25]:
#Calculate the minor/semiminor axis of orbits
data['minorA'] = np.sqrt((data['majorA'])**2 - (data['fDistance'])**2)
data['semiminorA'] = data['minorA']/2

In [26]:
#Mean motion of orbits
data['meanmotion'] = np.sqrt(GM/(data['semimajorA']**3))

In [27]:
#Finding the period in seconds
data['period_seconds'] = (2*np.pi)*(np.sqrt((data['semimajorA']**3)/(GM))) #gives more precise values
data['simple_seconds'] = data['period_minutes']*60
print("Radian approach\n")
print(data.period_seconds.head())
print("\nConverting from data\n")
print(data.simple_seconds.head())
print("\n Radian conversion gives more accurate figures")

Radian approach

0     5676.808714
1     5748.144371
2    86140.586331
3    85878.659303
4    86140.586331
Name: period_seconds, dtype: float64

Converting from data

0     5682.0
1     5754.0
2    86161.8
3    86166.0
4    86164.2
Name: simple_seconds, dtype: float64

 Radian conversion gives more accurate figures


In [28]:
#Calculating theoretical time  until longitudinal location for GEO orbits
print('Calculating directly via radians\n')
data['T_long'] = data['radians']*np.sqrt((data['semimajorA']**3)/GM)
print(data.T_long.head())

print("\n\nCalculating per degree longitude\n")
data['sec_per_longdegree'] = data['period_seconds']/360
print("\nSeconds/degree traveled")
print(data.sec_per_longdegree.head())
print("\nTimes longitudinal location")
data['time_till_longitude'] = data['longitude']*data['sec_per_longdegree']
print(data.time_till_longitude.head())

print("\nNo difference in accuracy")

Calculating directly via radians

0     2081.496529
1     3720.326773
2    17945.955486
3    67987.271948
4    34949.150110
Name: T_long, dtype: float64


Calculating per degree longitude


Seconds/degree traveled
0     15.768913
1     15.967068
2    239.279406
3    238.551831
4    239.279406
Name: sec_per_longdegree, dtype: float64

Times longitudinal location
0     2081.496529
1     3720.326773
2    17945.955486
3    67987.271948
4    34949.150110
Name: time_till_longitude, dtype: float64

No difference in accuracy


In [29]:
#Calculate mean anomaly at position X
data['meanAnom'] = data['meanmotion'] * data['time_till_longitude']

In [30]:
#Distance of earth from center of ellipse 
data['Efromcenter'] = data['semimajorA'] - data['perigeeR']

In [138]:
#Test function 
#def eccentricAnom(dm,de):
    #l = []
#def eccen(meanAnom, eccentricity):
def eccentricAnom(meanAnom, eccentricity):
    ea = meanAnom - ((meanAnom-(eccentricity*np.sin(meanAnom))-meanAnom)/(1-eccentricity*np.cos(meanAnom)))
    dif = ea - meanAnom
    return ea, dif
    
    #for i in range(len(dm)):
    #EA = eccen(dm,de)
    #while EA['dif'] > 0.0001:
        #print(EA['dif'])
        #EA = eccen(EA['ea'], de)
    #return EA['ea']
    #l.append(EA)
                
    #return l

In [119]:
# eccen(5.87286, 0.0501)# 0.8501)
data.tail()
print(eccentricAnom([2.98451302091, 3.490659],[0.00145, 0.000947]))# 0.8501)
#print(eccen(2.98451302091, 0.00145))# 0.8501)

(array([ 2.98473953,  3.49033539]), array([ 0.00022651, -0.00032361]))


In [128]:
#Calculate eccentric Anomaly at position X
def eccentricAnom(meanAnom, eccentricity):
    difference = pd.DataFrame()
    EA = meanAnom - ((meanAnom-(eccentricity*np.sin(meanAnom))-meanAnom)/(1-eccentricity*np.cos(meanAnom)))
    difference = EA - meanAnom
    for item, frame in difference.iteritems():
        print(frame)
        if (frame < 1.0e-03):
            return EA
        else:
            eccentricAnom(EA, eccentricity)
            
#data.meanAnom.apply(eccentest, axis=1)

In [104]:
print(data['meanAnom'].iloc[0]) 
print(data['eccentricity'].iloc[0])

2.30383461263
0.00145


In [122]:
data['eccentricAnom'],data['dif'] = eccentricAnom(data['meanAnom'], data['eccentricity'])
data.head(100)

,name,country_reg,country,owner,users,purpose,description,CLO,TOO,longitude,perigee_km,apogee_km,eccentricity,inclination,period_minutes,launch_mass,dry_mass,power_watts,launch_date,expected_lifetime_years,contractor,contractor_country,launch_site,launch_vehicle,COSPAR,NORAD,comments,launch_year,launch_month,launch_day,nickname,radians,apogeeR,perigeeR,majorA,semimajorA,fDistance,minorA,semiminorA,meanmotion,period_seconds,simple_seconds,T_long,sec_per_longdegree,time_till_longitude,meanAnom,Efromcenter,eccentricAnom,trueAnom,flightpath,distanceAt,cartX,cartY,trueAnomadj,dif
0,Aalto-1,NR (10/17),Finland,University of Aalto,Civil,Technology Development,NaN,LEO,NaN,132.00,497,517,0.001450,97.45,94.70,5.0,NaN,4.5,2017-06-23,2.00,University of Aalto,Finland,Satish Dhawan Space Centre,PSLV,2017-036L,42775,Technology development and education.,2017,6,23,NaN,2.303835,6888,6868,13756,6878.0,20,13755.985461,6877.992730,0.001107,5676.808714,5682.0,2081.496529,15.768913,2081.496529,2.303835,10.0,2.304911,3.072703,9.995604e-05,516.949375,-6861.685558,473.449925,6.145405,0.001077
1,AAUSat-4,NR (12/16),Denmark,University of Aalborg,Civil,Earth Observation,Automatic Identification System (AIS),LEO,Sun-Synchronous,233.00,442,687,0.017700,98.20,95.90,1.0,NaN,NaN,2016-04-25,NaN,University of Aalborg,Denmark,Guiana Space Center,Soyuz 2.1a,2016-025E,41460,Carries AIS system.,2016,4,25,NaN,4.066617,7058,6813,13871,6935.5,245,13868.836144,6934.418072,0.001093,5748.144371,5754.0,3720.326773,15.967068,3720.326773,4.066617,122.5,4.052630,3.073812,1.220342e-03,686.966326,-6919.574681,469.656942,6.147625,-0.013987
2,ABS-2,NR,Multinational,Asia Broadcast Satellite Ltd.,Commercial,Communications,NaN,GEO,NaN,75.00,35778,35793,0.000178,0.08,1436.03,6330.0,NaN,16000.0,2014-02-06,15.00,Space Systems/Loral,USA,Guiana Space Center,Ariane 5 ECA,2014-006A,39508,32 C-band 51 Ku-band and 6 Ka-band transponder...,2014,2,6,(Koreasat-8 ST-3),1.308997,42164,42149,84313,42156.5,15,84312.998666,42156.499333,0.000073,86140.586331,86161.8,17945.955486,239.279406,17945.955486,1.308997,7.5,1.309169,3.072615,1.227045e-05,35792.986006,-42056.251441,2905.547572,6.145230,0.000172
3,ABS-2A,NR (12/16),Multinational,Asia Broadcast Satellite Ltd.,Commercial,Communications,NaN,GEO,NaN,285.00,35700,35700,0.000000,0.00,1436.10,1800.0,NaN,NaN,2016-06-15,15.00,Boeing Satellite Systems,USA,Cape Canaveral,Falcon 9,2016-038A,41588,NaN,2016,6,15,NaN,4.974188,42071,42071,84142,42071.0,0,84142.000000,42071.000000,0.000073,85878.659303,86166.0,67987.271948,238.551831,67987.271948,4.974188,0.0,4.974188,3.072603,0.000000e+00,35700.000000,-41970.919181,2900.169668,6.145206,0.000000
4,ABS-3,Philippines,Multinational,Asia Broadcast Satellite Ltd.,Commercial,Communications,NaN,GEO,NaN,146.06,35769,35802,0.000391,0.05,1436.07,3775.0,1800.0,9000.0,1997-08-19,15.00,Space Systems/Loral,USA,Xichang Satellite Launch Center,Long March CZ3B,1997-042A,24901,Most powerful telecommunications satellite in ...,1997,8,19,(Agila 2 Mabuhay 1),2.549228,42173,42140,84313,42156.5,33,84312.993542,42156.496771,0.000073,86140.586331,86164.2,34949.150110,239.279406,34949.150110,2.549228,16.5,2.549446,3.072630,2.695364e-05,35801.943980,-42056.294087,2904.930051,6.145260,0.000218
5,ABS-3A,NR,Multinational,Asia Broadcast Satellite Ltd.,Commercial,Communications,NaN,GEO,NaN,357.00,35788,35803,0.000178,0.10,1436.00,2000.0,NaN,NaN,2015-03-02,15.00,Boeing Satellite Systems,USA,Cape Canaveral,Falcon 9,2015-010A,40424,Coverage of Americas Europe and Africa.,2015,3,2,NaN,6.230825,42174,42159,84333,42166.5,15,84332.998666,42166.499333,0.000073,86171.238435,86160.0,85453.144782,239.364551,85453.144782,6.230825,7.5,6.230816,3.072615,1.227045e-05,35802.987782,-42066.227661,2906.236801,6.145230,-0.000009
6,ABS-4,NR,Multinational,Asia Broadcast Satellite Ltd.,Commercial,Communications,NaN,GEO,NaN,75.00,35780,35793,0.000154,0.01,1436.10,4143.0,1700.0,7400.0,2004-03-13,12.00,Space Systems/Loral,USA,Cape Canaveral,Atlas 3,2004-007A,28184,Pur

In [150]:
l = []
for i,el in enumerate(data['meanAnom']):
    ea, dif = eccentricAnom(el,data['eccentricity'].iloc[i])
    while dif > 0.001:
        ea,dif = eccentricAnom(ea,data['eccentricity'].iloc[i])
    l.append(ea)

In [152]:
data['eccentricAnom'] = l

In [153]:
#checking 
data.loc[780]

name                                                                HYLAS 1 
country_reg                                                   United Kingdom
country                                                       United Kingdom
owner                                              Avanti Communications PLC
users                                                             Commercial
purpose                                                       Communications
description                                                              NaN
CLO                                                                      GEO
TOO                                                                      NaN
longitude                                                              326.5
perigee_km                                                             35770
apogee_km                                                              35803
eccentricity                                                        0.000391

In [154]:
#Calculate true Anomaly  at position X
data['trueAnom'] = np.arccos((np.cos(data['eccentricAnom'])-data['eccentricity'])/(1 - data['eccentricity']*np.cos(data['eccentricAnom'])))

In [155]:
#Adjusting for 360 degrees
data['trueAnomadj'] = data['trueAnom']*2

In [156]:
#Calculating flight path  at position X
data['flightpath'] = np.arctan(data['eccentricity']*np.sin(data['trueAnom'])/(1 + data['eccentricity']*np.cos(data['trueAnom'])))

In [157]:
#Calculating altitude at position X
data['distanceAt'] = (data['semimajorA'] * (1-data['eccentricity']*np.cos(data['eccentricAnom'])))-6371

In [168]:
#Converting to cartesian coordiantes
data['cartX'] = data['semimajorA']*np.cos(data['trueAnomadj']) #- data['Efromcenter']
data['cartY'] = data['semiminorA']*np.sin(data['trueAnomadj'])

In [169]:
data.head(20)

,name,country_reg,country,owner,users,purpose,description,CLO,TOO,longitude,perigee_km,apogee_km,eccentricity,inclination,period_minutes,launch_mass,dry_mass,power_watts,launch_date,expected_lifetime_years,contractor,contractor_country,launch_site,launch_vehicle,COSPAR,NORAD,comments,launch_year,launch_month,launch_day,nickname,radians,apogeeR,perigeeR,majorA,semimajorA,fDistance,minorA,semiminorA,meanmotion,period_seconds,simple_seconds,T_long,sec_per_longdegree,time_till_longitude,meanAnom,Efromcenter,eccentricAnom,trueAnom,flightpath,distanceAt,cartX,cartY,trueAnomadj,dif
0,Aalto-1,NR (10/17),Finland,University of Aalto,Civil,Technology Development,NaN,LEO,NaN,132.00,497,517,0.001450,97.45,94.70,5.0,NaN,4.5,2017-06-23,2.0,University of Aalto,Finland,Satish Dhawan Space Centre,PSLV,2017-036L,42775,Technology development and education.,2017,6,23,NaN,2.303835,6888,6868,13756,6878.0,20,13755.985461,6877.992730,0.001107,5676.808714,5682.0,2081.496529,15.768913,2081.496529,2.303835,10.0,2.380664,2.381664,0.001000,514.222480,350.202281,-6869.071453,4.763327,0.0
1,AAUSat-4,NR (12/16),Denmark,University of Aalborg,Civil,Earth Observation,Automatic Identification System (AIS),LEO,Sun-Synchronous,233.00,442,687,0.017700,98.20,95.90,1.0,NaN,NaN,2016-04-25,NaN,University of Aalborg,Denmark,Guiana Space Center,Soyuz 2.1a,2016-025E,41460,Carries AIS system.,2016,4,25,NaN,4.066617,7058,6813,13871,6935.5,245,13868.836144,6934.418072,0.001093,5748.144371,5754.0,3720.326773,15.967068,3720.326773,4.066617,122.5,4.052630,2.244466,0.013987,639.741808,-1536.921880,-6762.008729,4.488932,0.0
2,ABS-2,NR,Multinational,Asia Broadcast Satellite Ltd.,Commercial,Communications,NaN,GEO,NaN,75.00,35778,35793,0.000178,0.08,1436.03,6330.0,NaN,16000.0,2014-02-06,15.0,Space Systems/Loral,USA,Guiana Space Center,Ariane 5 ECA,2014-006A,39508,32 C-band 51 Ku-band and 6 Ka-band transponder...,2014,2,6,(Koreasat-8 ST-3),1.308997,42164,42149,84313,42156.5,15,84312.998666,42156.499333,0.000073,86140.586331,86161.8,17945.955486,239.279406,17945.955486,1.308997,7.5,1.309169,1.309341,0.000172,35783.559105,-36523.088472,21053.134843,2.618682,0.0
3,ABS-2A,NR (12/16),Multinational,Asia Broadcast Satellite Ltd.,Commercial,Communications,NaN,GEO,NaN,285.00,35700,35700,0.000000,0.00,1436.10,1800.0,NaN,NaN,2016-06-15,15.0,Boeing Satellite Systems,USA,Cape Canaveral,Falcon 9,2016-038A,41588,NaN,2016,6,15,NaN,4.974188,42071,42071,84142,42071.0,0,84142.000000,42071.000000,0.000073,85878.659303,86166.0,67987.271948,238.551831,67987.271948,4.974188,0.0,4.974188,1.308997,0.000000,35700.000000,-36434.554763,21035.500000,2.617994,0.0
4,ABS-3,Philippines,Multinational,Asia Broadcast Satellite Ltd.,Commercial,Communications,NaN,GEO,NaN,146.06,35769,35802,0.000391,0.05,1436.07,3775.0,1800.0,9000.0,1997-08-19,15.0,Space Systems/Loral,USA,Xichang Satellite Launch Center,Long March CZ3B,1997-042A,24901,Most powerful telecommunications satellite in ...,1997,8,19,(Agila 2 Mabuhay 1),2.549228,42173,42140,84313,42156.5,33,84312.993542,42156.496771,0.000073,86140.586331,86164.2,34949.150110,239.279406,34949.150110,2.549228,16.5,2.549446,2.549664,0.000218,35799.176838,15908.013961,-39039.791887,5.099329,0.0
5,ABS-3A,NR,Multinational,Asia Broadcast Satellite Ltd.,Commercial,Communications,NaN,GEO,NaN,357.00,35788,35803,0.000178,0.10,1436.00,2000.0,NaN,NaN,2015-03-02,15.0,Boeing Satellite Systems,USA,Cape Canaveral,Falcon 9,2015-010A,40424,Coverage of Americas Europe and Africa.,2015,3,2,NaN,6.230825,42174,42159,84333,42166.5,15,84332.998666,42166.499333,0.000073,86171.238435,86160.0,85453.144782,239.364551,85453.144782,6.230825,7.5,6.230816,0.052379,0.000009,35788.004653,41935.343194,4409.162372,0.104757,0.0
6,ABS-4,NR,Multinational,Asia Broadcast Satellite Ltd.,Commercial,Communications,NaN,GEO,NaN,75.00,35780,35793,0.000154,0.01,1436.10,4143.0,1700.0,7400.0,2004-03-13,12.0,Space Systems/Loral,USA,Cape Canaveral,Atlas 3,2004-007A,28184,Purchased by ABS in 2013.,2004,3,13,(ABS-2i MBSat Mobile Broa

In [170]:
data.to_csv('CSatellites.csv')

In [171]:
#Split data by type of orbit
LEO = data[data.CLO.isin(["LEO"])] #Low Earth orbit
GEO = data[data.CLO.isin(["GEO"])] #Geosynchronous
MEO_Elliptical = data[data.CLO.isin(["MEO", "Elliptical"])] #Medium earth orbit and Elliptical 





In [172]:
LEO.head()

,name,country_reg,country,owner,users,purpose,description,CLO,TOO,longitude,perigee_km,apogee_km,eccentricity,inclination,period_minutes,launch_mass,dry_mass,power_watts,launch_date,expected_lifetime_years,contractor,contractor_country,launch_site,launch_vehicle,COSPAR,NORAD,comments,launch_year,launch_month,launch_day,nickname,radians,apogeeR,perigeeR,majorA,semimajorA,fDistance,minorA,semiminorA,meanmotion,period_seconds,simple_seconds,T_long,sec_per_longdegree,time_till_longitude,meanAnom,Efromcenter,eccentricAnom,trueAnom,flightpath,distanceAt,cartX,cartY,trueAnomadj,dif
0,Aalto-1,NR (10/17),Finland,University of Aalto,Civil,Technology Development,NaN,LEO,NaN,132.0,497,517,0.00145,97.45,94.7,5.0,NaN,4.5,2017-06-23,2.0,University of Aalto,Finland,Satish Dhawan Space Centre,PSLV,2017-036L,42775,Technology development and education.,2017,6,23,NaN,2.303835,6888,6868,13756,6878.0,20,13755.985461,6877.992730,0.001107,5676.808714,5682.0,2081.496529,15.768913,2081.496529,2.303835,10.0,2.380664,2.381664,0.001000,514.222480,350.202281,-6869.071453,4.763327,0.0
1,AAUSat-4,NR (12/16),Denmark,University of Aalborg,Civil,Earth Observation,Automatic Identification System (AIS),LEO,Sun-Synchronous,233.0,442,687,0.01770,98.20,95.9,1.0,NaN,NaN,2016-04-25,NaN,University of Aalborg,Denmark,Guiana Space Center,Soyuz 2.1a,2016-025E,41460,Carries AIS system.,2016,4,25,NaN,4.066617,7058,6813,13871,6935.5,245,13868.836144,6934.418072,0.001093,5748.144371,5754.0,3720.326773,15.967068,3720.326773,4.066617,122.5,4.052630,2.244466,0.013987,639.741808,-1536.921880,-6762.008729,4.488932,0.0
17,Aeneas,USA,USA,Department of Homeland Security,Government,Technology Development,NaN,LEO,Equatorial,110.0,480,790,0.02210,0.02,97.4,3.0,NaN,NaN,2012-09-13,NaN,University of Southern California,USA,Vandenberg AFB,Atlas 5,2012-048C,38760,Demonsration program to track cargo containers...,2012,9,13,NaN,1.919862,7161,6851,14012,7006.0,310,14008.570377,7004.285188,0.001077,5836.012356,5844.0,1783.225998,16.211145,1783.225998,1.919862,155.0,3.096533,3.097518,0.000996,789.675445,6978.798409,-616.621905,6.195036,0.0
18,Aerocube 4.5A,USA,USA,Aerospace Corporation,Commercial,Technology Development,NaN,LEO,Non-Polar Inclined,24.0,495,791,0.02110,64.60,97.5,5.0,NaN,NaN,2012-09-13,NaN,Aerospace Corporation,USA,Vandenberg AFB,Atlas 5,2012-048K,38767,NaN,2012,9,13,NaN,0.418879,7162,6866,14028,7014.0,296,14024.876755,7012.438378,0.001075,5846.011234,5850.0,389.734082,16.238920,389.734082,0.418879,148.0,3.094397,3.095382,0.000996,790.830603,6984.065237,-647.180194,6.190763,0.0
19,Aerocube 4.5B,USA,USA,Aerospace Corporation,Commercial,Technology Development,NaN,LEO,Non-Polar Inclined,230.0,499,792,0.02090,64.60,97.6,5.0,NaN,NaN,2012-09-13,NaN,Aerospace Corporation,USA,Vandenberg AFB,Atlas 5,2012-048L,38768,NaN,2012,9,13,NaN,4.014257,7163,6870,14033,7016.5,293,14029.940841,7014.970420,0.001074,5849.137053,5856.0,3736.948673,16.247603,3736.948673,4.014257,146.5,3.998459,2.300416,0.015798,741.524358,-781.113540,-6971.365550,4.600833,0.0


In [173]:
#Move cislunar satellite into Elliptical MEO group
cis = data[data.TOO.isin(["Cislunar"])]
frames = [MEO_Elliptical, cis]
MEO_Elliptical = pd.concat(frames)

In [174]:
MEO_Elliptical.TOO.value_counts()

Non-Polar Inclined       85
Molniya                  16
Equatorial               12
Deep Highly Eccentric     9
Cislunar                  1
Name: TOO, dtype: int64

In [175]:
#Split Low earth orbit into sun-synchronous and other
LEO_sun = LEO[LEO.TOO.isin(["Sun-Synchronous"])]

In [176]:
LEO_other = LEO[LEO.TOO.isin(["Non-Polar Inclined", "Polar", "Equatorial", "Elliptical"])]

In [177]:
LEO_sun.to_csv("LEO_sun.csv")
LEO_other.to_csv("LEO_other.csv")
GEO.to_csv("GEO.csv")
MEO_Elliptical.to_csv("MEO_Elliptical.csv")

In [49]:
min(data.launch_year)

1974

In [178]:
max(data.launch_year)

2017

In [196]:
#pd.value_counts(data.owner[data.country == "Russia"])
pd.value_counts(data.country)

USA                                  786
China                                203
Russia                               138
Japan                                 65
Multinational                         65
India                                 50
United Kingdom                        44
ESA                                   38
Canada                                33
Germany                               26
Luxembourg                            18
Spain                                 13
France                                12
Brazil                                12
Israel                                12
South Korea                           11
Italy                                 11
Netherlands                           11
Saudi Arabia                          11
Argentina                             11
Australia                              9
Turkey                                 9
Norway                                 8
Singapore                              8
Indonesia       

In [50]:
##Take previous formulae and create function to calculate x and y coordinates for every point in the flightpath
def orbits(sec_per_degree, meanmotion):
    counter = 0
    while counter <361:
        meanAnom = []
        meanAnom[counter] = meanmotion * (sec_per_degree*counter)
        counter +=1
    return meanAnom

In [51]:
orbitPaths = DataFrame(orbits(data['sec_per_longdegree'], data['meanmotion']))
orbitPaths

IndexError: list assignment index out of range

In [ ]:
        eccenAnom = eccentricAnom(meanAnom,eccentricity)
        trueAnom = np.arccos((np.cos(eccenAnom)-eccentricity)/(1 - eccentricity*np.cos(eccenAnom)))
        distanceAt = (semimajorA * (1-eccentricity*np.cos(eccenAnom)))-6371
        trueAnomadj = trueAnom*2

In [ ]:
orbitPaths['cartX'] = semimajorA*np.cos(trueAnomadj) #- data['Efromcenter']
orbitPaths['cartY'] = semiminorA*np.sin(trueAnomadj)